In [1]:
import pandas as pd
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.master("local[1]").appName('ML_project.com').getOrCreate()

In [4]:
df = spark.read.csv("./main_data/clickstream.csv")

In [5]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)



In [10]:
df.show()

+--------------------+------+-------------------+-------+
|                 _c0|   _c1|                _c2|    _c3|
+--------------------+------+-------------------+-------+
|             user_id|cat_id|          timestamp|new_uid|
|000143baebad4467a...|   165|2021-01-30 20:08:12|1873448|
|000143baebad4467a...|   165|2021-01-31 20:06:29|1873448|
|000143baebad4467a...|   308|2021-01-31 20:12:00|1873448|
|000143baebad4467a...|   931|2021-01-31 22:12:00|1873448|
|000143baebad4467a...|   931|2021-02-01 16:57:00|1873448|
|000143baebad4467a...|   931|2021-02-01 17:39:00|1873448|
|000143baebad4467a...|   503|2021-02-01 17:41:50|1873448|
|000143baebad4467a...|   535|2021-02-01 19:02:00|1873448|
|000143baebad4467a...|   535|2021-02-01 19:43:00|1873448|
|000143baebad4467a...|   931|2021-02-01 19:51:00|1873448|
|000143baebad4467a...|   535|2021-02-01 20:32:00|1873448|
|000143baebad4467a...|   165|2021-02-02 18:02:45|1873448|
|000143baebad4467a...|   503|2021-02-02 19:42:38|1270185|
|000143baebad4